TST(time series transformer)の定義と学習  
time series transformerについて  
https://huggingface.co/blog/time-series-transformers  


In [11]:
# デコーダついてるけど使える？
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel

# Initializing a Time Series Transformer configuration with 12 time steps for prediction
configuration = TimeSeriesTransformerConfig(prediction_length=12)

# Randomly initializing a model (with random weights) from the configuration
model = TimeSeriesTransformerModel(configuration)

# Accessing the model configuration
configuration = model.config

print(model)

TimeSeriesTransformerModel(
  (scaler): TimeSeriesMeanScaler()
  (encoder): TimeSeriesTransformerEncoder(
    (value_embedding): TimeSeriesValueEmbedding(
      (value_projection): Linear(in_features=9, out_features=64, bias=False)
    )
    (embed_positions): TimeSeriesSinusoidalPositionalEmbedding(24, 64)
    (layers): ModuleList(
      (0-1): 2 x TimeSeriesTransformerEncoderLayer(
        (self_attn): TimeSeriesTransformerAttention(
          (k_proj): Linear(in_features=64, out_features=64, bias=True)
          (v_proj): Linear(in_features=64, out_features=64, bias=True)
          (q_proj): Linear(in_features=64, out_features=64, bias=True)
          (out_proj): Linear(in_features=64, out_features=64, bias=True)
        )
        (self_attn_layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (activation_fn): GELUActivation()
        (fc1): Linear(in_features=64, out_features=32, bias=True)
        (fc2): Linear(in_features=32, out_features=64, bias=True)
     

In [21]:
import torch
import torch.nn as nn
from transformers import TimeSeriesTransformerConfig, TimeSeriesTransformerModel

class TimeSeriesTransformerForClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.time_series_transformer = TimeSeriesTransformerModel(config)
        self.classifier = nn.Linear(config.d_model, 8)
    
    def forward(self, past_values, past_time_features, past_observed_mask):
        transformer_output = self.time_series_transformer(
            past_values=past_values,
            past_time_features=past_time_features,
            past_observed_mask=past_observed_mask
        )
        hidden_states = transformer_output.last_hidden_state
        pooled_output = hidden_states.mean(dim=1)
        logits = self.classifier(pooled_output)
        return logits


In [17]:
# コンフィグの設定
config = TimeSeriesTransformerConfig(prediction_length=12)
model = TimeSeriesTransformerForClassification(config)
print(model)

TimeSeriesTransformerForClassification(
  (time_series_transformer): TimeSeriesTransformerModel(
    (scaler): TimeSeriesMeanScaler()
    (encoder): TimeSeriesTransformerEncoder(
      (value_embedding): TimeSeriesValueEmbedding(
        (value_projection): Linear(in_features=9, out_features=64, bias=False)
      )
      (embed_positions): TimeSeriesSinusoidalPositionalEmbedding(24, 64)
      (layers): ModuleList(
        (0-1): 2 x TimeSeriesTransformerEncoderLayer(
          (self_attn): TimeSeriesTransformerAttention(
            (k_proj): Linear(in_features=64, out_features=64, bias=True)
            (v_proj): Linear(in_features=64, out_features=64, bias=True)
            (q_proj): Linear(in_features=64, out_features=64, bias=True)
            (out_proj): Linear(in_features=64, out_features=64, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=

In [39]:
config = TimeSeriesTransformerConfig(
    prediction_length=12,
    #context_length=500,
    num_time_features=9,
)
model = TimeSeriesTransformerForClassification(config)

In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, TensorDataset, Dataset

from transformers import DistilBertModel, DistilBertTokenizer, TrainingArguments, Trainer, DistilBertConfig
from transformers.modeling_outputs import SequenceClassifierOutput

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm


#import utils

# For reproducibility
np.random.seed(42)
torch.manual_seed(42) # 乱数生成シード
cudnn.benchmark = True

# Grab a GPU if there is one
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using {} device: {}".format(device, torch.cuda.current_device()))
else:
    device = torch.device("cpu")
    print("Using {}".format(device))

Using cuda device: 0


In [41]:
fold = "../../train_raw_npy/"
acc_xyz = np.load(f"{fold}acc_xyz_back80000.npy")
gyr_xyz = np.load(f"{fold}gyr_xyz_back80000.npy")
mag_xyz = np.load(f"{fold}mag_xyz_back80000.npy")
label = np.load(f"{fold}sampled_label_back80000.npy")
xyz = np.concatenate([acc_xyz, gyr_xyz, mag_xyz], axis=1)
del acc_xyz
del gyr_xyz
del mag_xyz
print(xyz.shape)
print(label.shape)

(79999, 9, 500)
(79999,)


In [42]:
from sklearn.preprocessing import StandardScaler

# データを2Dに変換して標準化
scaler = StandardScaler()
xyz_reshaped = xyz.transpose(0, 2, 1).reshape(-1, xyz.shape[1])  # (196072*500, 9)
scaled_xy = scaler.fit_transform(xyz_reshaped)  # 標準化
scaled_xy = scaled_xy.reshape(xyz.shape[0], xyz.shape[2], xyz.shape[1])  # 元の形に戻すが、(196072, 500, 9) の形状に
print(scaled_xy.shape)  # (196072, 500, 9)

(79999, 500, 9)


In [43]:
#おしり3万データを使う  
scaled_xyz = scaled_xy[-40000:-10000,:,:]
label_sm = label[-40000:-10000]
print(scaled_xyz.shape)
print(label_sm.shape)

(30000, 500, 9)
(30000,)


In [44]:
import numpy as np
from sklearn.model_selection import train_test_split

# ランダムサンプリング
train_xyz, test_xyz, train_label, test_label = train_test_split(scaled_xyz, label_sm, test_size=0.2,random_state=40)

# 結果の確認
print(f"Train data shape: {train_xyz.shape}")
print(f"Test data shape: {test_xyz.shape}")
print(f"Train label shape: {train_label.shape}")
print(f"Test label shape: {test_label.shape}")
print(pd.Series(train_label).value_counts())
print(pd.Series(test_label).value_counts())

Train data shape: (24000, 500, 9)
Test data shape: (6000, 500, 9)
Train label shape: (24000,)
Test label shape: (6000,)
1.0    4233
4.0    3919
7.0    3893
6.0    3585
2.0    3009
5.0    2965
8.0    1445
3.0     951
Name: count, dtype: int64
1.0    1043
7.0     964
4.0     958
6.0     922
5.0     773
2.0     721
8.0     373
3.0     246
Name: count, dtype: int64


In [45]:
# ラベルを0~7クラス分類とする
print(train_label)
train_label -=1
print(train_label)
print(test_label)
test_label -=1
print(test_label)

[4. 6. 4. ... 5. 5. 5.]
[3. 5. 3. ... 4. 4. 4.]
[7. 4. 3. ... 6. 6. 7.]
[6. 3. 2. ... 5. 5. 6.]


In [46]:
train_xyz = torch.from_numpy(train_xyz)
train_label = torch.from_numpy(train_label)
test_xyz = torch.from_numpy(test_xyz)
test_label = torch.from_numpy(test_label)

In [47]:
# データセットの準備

class SensorDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sensor_data = torch.tensor(self.data[idx], dtype=torch.float)
        labels = torch.tensor(self.labels[idx], dtype=torch.long)

        attention_mask = torch.ones(sensor_data.shape[0], dtype=torch.long)  # (500,) の形状になるように

        return {
            "input_ids": sensor_data,
            "attention_mask": attention_mask,
            "labels": labels
        }

# データセットとデータローダの作成
train_dataset = SensorDataset(train_xyz, train_label)
eval_dataset = SensorDataset(test_xyz, test_label)

In [48]:
# コンフィグの設定
config = TimeSeriesTransformerConfig(prediction_length=12)
model = TimeSeriesTransformerForClassification(config)
print(model)

TimeSeriesTransformerForClassification(
  (time_series_transformer): TimeSeriesTransformerModel(
    (scaler): TimeSeriesMeanScaler()
    (encoder): TimeSeriesTransformerEncoder(
      (value_embedding): TimeSeriesValueEmbedding(
        (value_projection): Linear(in_features=9, out_features=64, bias=False)
      )
      (embed_positions): TimeSeriesSinusoidalPositionalEmbedding(24, 64)
      (layers): ModuleList(
        (0-1): 2 x TimeSeriesTransformerEncoderLayer(
          (self_attn): TimeSeriesTransformerAttention(
            (k_proj): Linear(in_features=64, out_features=64, bias=True)
            (v_proj): Linear(in_features=64, out_features=64, bias=True)
            (q_proj): Linear(in_features=64, out_features=64, bias=True)
            (out_proj): Linear(in_features=64, out_features=64, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=

In [61]:
# トレーニング設定
training_args = TrainingArguments(
    output_dir="./results_distilBERT",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=16,
    save_steps=0,
    save_total_limit=None,
    #logging_dir = "./logs_distilBERT",
    evaluation_strategy="steps",
    eval_steps=1_000,
    logging_steps=100,
    max_grad_norm=1.0, # 勾配クリッピング
    fp16=True, # fp16計算
    #weight_decay=0.01
)

# トレーナーの定義
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# トレーニングの実行
trainer.train()

  0%|          | 0/15000 [00:00<?, ?it/s]

C:\Users\kaiha\AppData\Local\Temp\ipykernel_40656\1253163707.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sensor_data = torch.tensor(self.data[idx], dtype=torch.float)
C:\Users\kaiha\AppData\Local\Temp\ipykernel_40656\1253163707.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(self.labels[idx], dtype=torch.long)


ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: past_values,past_time_features,past_observed_mask,label,label_ids.

In [59]:
from huggingface_hub import hf_hub_download
import torch
from transformers import TimeSeriesTransformerModel

file = hf_hub_download(
    repo_id="hf-internal-testing/tourism-monthly-batch", filename="train-batch.pt", repo_type="dataset"
)
batch = torch.load(file)

print(len(batch.keys()))

8


もしかすると８個のデータがないといけないのか？そんなことある？